# Python Bindings

If we want to use C or C++ library in your Python program. Python Binding allows us to do this.
We get strength of both language (C and Python) or (C++ and Python). Below can be cases to use Python Binding.:

1)You already have a large, tested, stable library written in C++ that you’d like to take advantage of in Python. This may be a communication library or a library to talk to a specific piece of hardware. What it does is unimportant.

2)You want to speed up a particular section of your Python code by converting a critical section to C. Not only does C have faster execution speed, but it also allows you to break free from the limitations of the GIL, provided you’re careful.

3)You want to use Python test tools to do large-scale testing of their systems.


## Marshalling Data Types

The process of transforming the memory representation of an object to a data format suitable for storage or transmission.(Wikipedia)

1) In C, C++ we store data in a way which is altogether different in Python.(In Python every thing is object even Int)

2) Even C and C++ store data in different ways.

3) To transfer data between Python and C or C++ we need to perform some operation. This operation is called Marshalling.

Marshalling is what the Python bindings are doing when they prepare data to move it from Python to C or vice versa.

C stores data in the most compact form in memory possible. If you use an uint8_t, then it will only use 8 bits of memory total.

In Python, on the other hand, everything is an object. This means that each integer uses several bytes in memory. How many will depend on which version of Python you’re running, your operating system, and other factors. This means that your Python bindings will need to convert a C integer to a Python integer for each integer passed across the boundary.

## Understanding Mutable and Immutable Values

In C, all parameters are pass-by-value. If you want to allow a function to change a variable in the caller, then you need to pass a pointer to that variable.

Python won’t give you a pointer to an object easily, so this just doesn’t work.

If you want to modify a Python object in C, then you’ll need to take extra steps to achieve this.

## Managing Memory

C and Python manage memory differently.

In C, the developer must manage all memory allocations and ensure they’re freed once and only once. 
Python takes care of this for you using a garbage collector.

We’ll need to be aware of where the memory for each object was allocated and ensure that it’s only freed on the same side. 

Python data must be cleaned at Python side and C data must be cleaned at C side.

## ctypes

Library in Python to create Python Bindings.

It provides a low-level toolset for loading shared libraries and marshalling data between Python and C.


### Calling the Function

To create your Python bindings in ctypes, you need to do these steps:

1) Load your library.

2) Wrap some of your input parameters.

3) Tell ctypes the return type of your function.

### ctypes Strengths

The biggest advantage ctypes has over the other tools you’ll examine here is that it’s built into the standard library.  
It also requires no extra steps, as all of the work is done as part of your Python program.

### ctypes Weakness

The concepts used are low-level, which makes exercises like the one you just did manageable.  
More complex tasks grow cumbersome with the lack of automation. Here comes CFFI.

## CFFI

CFFI is the C Foreign Function Interface for Python. It takes a more automated approach to generate Python bindings. CFFI has multiple ways in which you can build and use your Python bindings. There are two different options to select from, which gives you four possible modes:

ABI vs API: API mode uses a C compiler to generate a full Python module, whereas ABI mode loads the shared library and interacts with it directly. Without running the compiler, getting the structures and parameters correct is error-prone. The documentation heavily recommends using the API mode.

in-line vs out-of-line: The difference between these two modes is a trade-off between speed and convenience:

In-line mode compiles the Python bindings every time your script runs. This is convenient, as you don’t need an extra build step. It does, however, slow down your program.
Out-of-line mode requires an extra step to generate the Python bindings a single time and then uses them each time the program is run. This is much faster, but that may not matter for your application.
For this example, you’ll use the API out-of-line mode, which produces the fastest code and, in general, looks similar to other Python bindings you’ll create later in this tutorial.

### CFFI Strengths

 CFFI scales to larger projects much better than ctypes due to automation of much of the function wrapping.  
 ctypes allows you to load a pre-existing C library directly into your Python program. CFFI, on the other hand,    creates a new Python module that can be loaded like other Python modules.  
 With the out-of-line-API method you used above, the time penalty for creating the Python bindings is done once when you build it and doesn’t happen each time you run your code. For small programs, this might not be a big deal, but CFFI scales better to larger projects in this way, as well.

### CFFI Weakness

ctypes and CFFI Works for C library.  
C++ libraries require a good deal of work to use. Here comes PyBind11

## PyBind11

PyBind11 takes a quite different approach to create Python bindings. In addition to shifting the focus from C to C++, it also uses C++ to specify and build the module, allowing it to take advantage of the metaprogramming tools in C++. Like CFFI, the Python bindings generated from PyBind11 are a full Python module that can be imported and used directly.

PyBind11 is modeled after the Boost::Python library and has a similar interface. It restricts its use to C++11 and newer, however, which allows it to simplify and speed things up compared to Boost, which supports everything.

### PyBind11 Strengths

PyBind11 is focused on C++ instead of C  
It supports classes. 
It handles polymorphic subclassing.  
It allows you to add dynamic attributes to objects from Python and many other tools, which would be quite difficult to do from the C-based tools(ctypes and CFFI)

### PyBind11 Weakness

Getting the installation and build correct can be a bit complex.  
PyBind11 requires that you use at least C++11 or newer. This is unlikely to be a big restriction for most projects.  
The extra code you need to write to create the Python bindings is in C++ and not Python.(Here comes Cython)

## Cython

The approach Cython takes to creating Python bindings uses a Python-like language to define the bindings and then generates C or C++ code that can be compiled into the module.  
There are several methods for building Python bindings with Cython. The most common one is to use setup from distutils.

### Cython Strengths

It provide a deep level of control when creating Python bindings for either C or C++.  
It provides a Python-esque method for writing code that manually controls the GIL, which can significantly speed up certain types of problems.

### Cython Weekness

That Python-esque language is not quite Python, however, so there’s a slight learning curve when you’re coming up to speed in figuring out which parts of C and Python fit where.

## PyBindGen

PyBindGen generates Python bindings for C or C++ and is written in Python. It’s targeted at producing readable C or C++ code, which should simplify debugging issues. It wasn’t clear if this has been updated recently, as the documentation lists Python 3.4 as the latest tested version. There have been yearly releases for the last several years, however.

## Boost.Python

Boost.Python has an interface similar to PyBind11, which you saw above. That’s not a coincidence, as PyBind11 was based on this library! Boost.Python is written in full C++ and supports most, if not all, versions of C++ on most platforms. In contrast, PyBind11 restricts itself to modern C++.

## SIP

SIP is a toolset for generating Python bindings that was developed for the PyQt project. It’s also used by the wxPython project to generate their bindings, as well. It has a code generation tool and an extra Python module that provides support functions for the generated code.

## Cppyy

cppyy is an interesting tool that has a slightly different design goal than what you’ve seen so far.

The original idea behind cppyy (going back to 2001), was to allow Python programmers that live in a C++ world access to those C++ packages, without having to touch C++ directly (or wait for the C++ developers to come around and provide bindings).

## Shiboken


Shiboken is a tool for generating Python bindings that’s developed for the PySide project associated with the Qt project. While it was designed as a tool for that project, the documentation indicates that it’s neither Qt- nor PySide-specific and is usable for other projects.

## SWIG

SWIG is a different tool than any of the others listed here. It’s a general tool used to create bindings to C and C++ programs for many other languages, not just Python. This ability to generate bindings for different languages can be quite useful in some projects. It, of course, comes with a cost as far as complexity is concerned.